In [1]:
import datajoint as dj

In [2]:
import ephys
import matplotlib.pyplot as plt
%matplotlib inline

Connecting daveliu@mesoscale-activity.datajoint.io:3306


In [ ]:
def dropdbs():
   print('dropping databases')
   for a in range(3):
       for d in ['ingest', 'ccf', 'ephys', 'experiment', 'lab']:
           try:
               schema = dj.schema(dj.config['%s.database' % d], locals())
               schema.drop(force=True)
           except Exception as e:
               print('error dropping {d} in attempt {a}: {e}'
                     .format(d=d, a=a, e=str(e)))
               pass

In [ ]:
dj.ERD(ephys)

In [ ]:
schema = dj.schema(dj.config['experiment.database'], locals())
schema.drop(force=True)

In [ ]:
dropdbs()

In [6]:
person=ephys.experiment.lab.Person()
#person.insert1(('daveliu', 'Dave Liu'))
person

username,fullname
daveliu,Dave Liu


In [5]:
rig=ephys.experiment.lab.Rig()
#rig.insert1(('TRig2', 'Training rig 2'))
#rig.insert1(('TRig1', 'Training rig 1'))
#rig.insert1(('RRig', 'Recording rig'))
rig

rig,rig_description
RRig,Recording rig
TRig1,Training rig 1
TRig2,Training rig 2


In [ ]:
animalWaterRestriction=ephys.experiment.lab.AnimalWaterRestriction()
#animalWaterRestriction.insert1((123456, 'tw5'))
animalWaterRestriction

In [ ]:
dj.ERD(ephys.experiment)

In [7]:
import ingest
rigDataPath = ingest.RigDataPath()
rigDataFile = ingest.RigDataFile()
rigDataFileIngest = ingest.RigDataFileIngest()

In [8]:
#importedSessionFile.drop()
rigDataFile.populate()

In [9]:
rigDataFileIngest.populate()

ValueError: too many values to unpack (expected 5)

In [10]:
files= rigDataFile.fetch()

In [14]:
len(files)

1256

In [16]:
files[7]

('TRig1', '67\\yes_no_ADS_autoTrain\\Session Data\\67_yes_no_ADS_autoTrain_20170728_100039.mat')

In [15]:
session=ephys.experiment.Session()
session

animal Janelia ANM ID (6 digits),session session number,session_date,username,rig
399752,1,2017-12-19,daveliu,RRig
399752,2,2017-12-20,daveliu,RRig
399752,3,2017-12-21,daveliu,RRig
399752,4,2017-12-22,daveliu,RRig
399752,5,2017-12-23,daveliu,RRig
399752,6,2017-12-24,daveliu,RRig


In [ ]:
trialNote = ephys.experiment.TrialNote()

In [ ]:
import numpy as np

In [ ]:
behaviorTrial = ephys.experiment.BehaviorTrial()

In [ ]:
outcome = (behaviorTrial & 'animal = 399752').fetch('outcome', order_by='session')
early_lick = (behaviorTrial & 'animal = 399752').fetch('early_lick', order_by='session')

In [ ]:
data = np.zeros(len(outcome))
data1 = np.zeros(len(early_lick))
data2 = np.zeros(len(outcome))

In [ ]:
data[outcome=='hit'] = 1
data1[early_lick=='early'] = 1
data2[outcome=='ignore'] = 1

In [ ]:
trial_note = (trialNote & 'animal = 123456').fetch('trial_note', order_by='session')
results = list(map(int, trial_note))
protDiff = np.diff(results)
protChange = np.where(protDiff != 0)

In [ ]:
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'valid')
    return sma

In [ ]:
yMA = movingaverage(data,200)
plt.plot(yMA)
plt.xlabel('Trials')
plt.ylabel('Proportion Correct')
for i in range(0, len(protChange[0])):
    plt.axvline(protChange[0][i], linewidth=4, color='r')


In [ ]:
yMA = movingaverage(data1,200)
plt.plot(yMA)
plt.xlabel('Trials')
plt.ylabel('Early Licks')
for i in range(0, len(protChange[0])):
    plt.axvline(protChange[0][i], linewidth=4, color='r')


In [ ]:
yMA = movingaverage(data2,20)
plt.plot(yMA)
plt.xlabel('Trials')
plt.ylabel('Ignores')
for i in range(0, len(protChange[0])):
    plt.axvline(protChange[0][i], linewidth=4, color='r')
plt.savefig('')

In [ ]:
len(outcome)